# Understanding Optimisers - Example Implementation

Optimization plays a crucial role in decision-making processes across various industries and disciplines. Whether it's allocating resources efficiently, scheduling tasks, or selecting projects to maximize benefits under constraints, optimization techniques provide systematic and quantitative methods to arrive at the best possible solutions.

In this notebook, we embark on a practical journey to understand how optimization can be applied to a real-world scenario using Python. Specifically, we will:

- **Simulate a Business Automation Scenario:** We'll create a set of business-as-usual (BAU) process automation projects, each with associated benefits such as hours saved per month, costs like hours required to implement, and specific skill requirements.
- **Utilize Advanced Data Structures:** We'll leverage Python's dataclasses to define clear and efficient data structures for our projects and team members, enhancing code readability and maintainability.
- **Implement an Optimization Model with `PuLP`:** We'll formulate and solve an optimization problem using PuLP, a powerful yet user-friendly linear programming library in Python. The goal is to select the optimal set of projects and assign team members effectively, maximizing the total hours saved while respecting various constraints.

## Objectives of the Notebook

  - Provide a step-by-step guide on how to model and solve an optimization problem in Python, suitable for learners new to optimization or Python programming.
  - Show how optimization techniques can be directly applied to manage resources and project selection in a business context.
  - Emphasize good coding practices by using modern Python features like dataclasses, and include comprehensive explanations through markdown cells.

## Scenario Overview

Imagine you are managing a team responsible for automating repetitive processes within an organization. You have:

- A list of potential automation projects, each requiring certain skills to implement and promising a specific amount of time saved per month once completed.
- A team of members with varying skills, Python proficiency levels (Basic, Intermediate, Advanced), and limited availability (hours they can allocate until the end of the financial year).
- Constraints such as only being able to run a maximum of two projects concurrently due to resource limitations.

Our Optimiser must:

- Select the best combination of projects to undertake that maximizes the total hours saved per month.
- Assign team members to projects in a way that respects their skill sets, availability, and the project's skill requirements.

## Key Concepts and Tools

### Optimization with PuLP

Our Optimiser is based on the commercial `Gurobi` engine - we're using `PuLP` as a FOSS stand-in for this code.

`PuLP` is an open-source linear programming (LP) library in Python that allows you to:

- Define decision variables, objective functions, and constraints in a mathematical optimization problem.
- Use built-in solvers like `CBC` (Coin-or branch and cut) to find optimal solutions.

By utilizing `PuLP`, we'll translate our business problem into a mathematical model, enabling us to compute the optimal project selection and team assignments programmatically.


### Data Classes (dataclasses)

Introduced in Python 3.7, dataclasses provide a decorator and functions for automatically adding special methods to user-defined classes. They simplify class creation by:

- Automatically generating methods like `__init__()` and `__repr__()`.
- Making the code cleaner and more focused on the actual data being stored.

We'll use dataclasses to define our `Project` and `TeamMember` classes, ensuring our code is concise and easy to understand.

### Constraints and Decision Variables

In our optimization model, we'll consider several constraints:

- Maximum Concurrent Projects: Limit the number of projects running at the same time.
- Skill Requirements: Ensure team members assigned to projects have the necessary skills and Python proficiency levels.
- Availability: Team members cannot be assigned to more hours than they have available.
- Assignment Constraints: Team members can only be assigned to selected projects.

Our decision variables will include:

- Project Selection Variables: Binary variables indicating whether a project is selected.
- Team Assignment Variables: Binary variables indicating whether a team member is assigned to a project.

## Structure of the Notebook

The notebook is organized into the following sections:

- **Environment Setup:** Installing and importing necessary libraries.
- **Data Definition:** Creating dataclasses for Project and TeamMember, and populating them with sample data.
- **Problem Formulation:** Defining the optimization problem's objective function and constraints using PuLP.
- **Model Implementation:** Translating the mathematical model into code.
 -**Solution and Interpretation:** Solving the optimization problem and interpreting the results.
- **Conclusion:** Summarizing what we've learned and potential next steps.

## Learning Outcomes

By the end of this notebook, you should be able to:

- Understand how to model an optimization problem in Python using PuLP.
- Utilize dataclasses to create clean and efficient data structures.
- Translate a real-world scenario into a mathematical optimization model.
- Interpret the results of the optimization and understand their implications in a business context.

## Prerequisites

To fully benefit from this notebook, you should have:

- Basic knowledge of Python programming.
- Familiarity with concepts in linear programming and optimization (helpful but not required, see section below for a crash course).
- An understanding of business resource allocation challenges.

## Let's Get Started!

In the following sections, we'll dive into the practical implementation, starting with setting up our environment and defining our data structures. Feel free to modify the sample data and experiment with different scenarios to deepen your understanding.

In [1]:
struct Project
    bau_project::String
    hours_to_complete::Int64
    monthly_hours_saved::Float64
    skills_needed::Array
end

struct TeamMember
    name::String
    skills::Array
    max_hours_available::Int16
end

In [2]:
sec_del = Project("Sec Del Reporting", 240, 20, ["APIs", "Advanced Python", "Excel"])
narm_kpis = Project("NARM KPI Reports", 160, 5, ["Basic Python", "Excel", "SQL"])
im_wos = Project("I&M Work Order Generation", 160, 10, ["APIs", "Intermediate Python", "Excel", "SQL"])
interventions = Project("Interventions Generation", 40, 2, ["APIs", "Intermediate Python", "Excel"])
pulse = Project("PULSE Report Generation", 80, 10, ["Intermediate Python"])

Project("PULSE Report Generation", 80, 10.0, ["Intermediate Python"])

In [3]:
a = TeamMember("a", ["SQL", "Advanced Python", "Intermediate Python", "Basic Python", "APIs", "Excel"], 75)
b = TeamMember("b", ["SQL", "Intermediate Python", "Basic Python", "Excel"], 140)
c = TeamMember("c", ["SQL", "Excel"], 210)
d = TeamMember("d", ["Basic Python", "Excel"], 70)
e = TeamMember("e", ["SQL", "Advanced Python", "Intermediate Python", "Basic Python", "Excel"], 160)

TeamMember("e", ["SQL", "Advanced Python", "Intermediate Python", "Basic Python", "Excel"], 160)

In [4]:
projects = [sec_del, narm_kpis, im_wos, interventions, pulse]
team_members = [a, b, c, d, e]

5-element Vector{TeamMember}:
 TeamMember("a", ["SQL", "Advanced Python", "Intermediate Python", "Basic Python", "APIs", "Excel"], 75)
 TeamMember("b", ["SQL", "Intermediate Python", "Basic Python", "Excel"], 140)
 TeamMember("c", ["SQL", "Excel"], 210)
 TeamMember("d", ["Basic Python", "Excel"], 70)
 TeamMember("e", ["SQL", "Advanced Python", "Intermediate Python", "Basic Python", "Excel"], 160)

In [6]:
all_hours_to_complete = []
for proj in projects
    append!(all_hours_to_complete, proj.hours_to_complete)
end

skills_matrix = Matrix()

for person in team_members
    can_work_on_project = []
    for proj in projects
        for skill in person.skills
            for skill_needed in proj.skills_needed
                if skill in skill_neede
